In [2]:
import pandas as pd, requests, numpy as np
import urllib.request
import bar_chart_race as bcr
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
##LOAD Town COVIDATA [MOH-IL]
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=8a21d39d-91e3-40db-aca1-f73f7ab1df69&limit=150000'
json = requests.get(url).json()
df = pd.DataFrame(json['result']['records'])


In [4]:
##load town data [CBS-IL]
url='https://www.cbs.gov.il/he/publications/doclib/2019/ishuvim/bycode2019.xlsx'
cbs_town = pd.read_excel(url)


In [5]:
##lower case column name
df = df.rename(columns=lambda x:x.lower())

## use 10 instead of less than
df = df.replace('<15','10')

## cols data types
cols = ['city_code','cumulative_verified_cases',
       'cumulated_recovered', 'cumulated_deaths', 'cumulated_number_of_tests',
       'cumulated_number_of_diagnostic_tests']
for x in cols:
    df[x] = df[x].astype('int')
df['date']=pd.to_datetime(df.date)
# df.dtypes
# df.head()

In [6]:
## town_df
town_df = cbs_town[['סמל יישוב', 'סך הכל אוכלוסייה 2019']].rename(columns={'סמל יישוב' : 'city_code',
                                                                           'סך הכל אוכלוסייה 2019' : 'pop'})

##convert population na to 0
town_df['pop'] = town_df['pop'].fillna(0).astype('int')

##merge to main COVIDF
covidf = df.merge(town_df, on='city_code')

In [7]:
# covidf_town
covidf_town = covidf[['city_code','city_name','pop']].drop_duplicates().sort_values(by='pop', ascending=False).reset_index(drop=True)

# remove tribes (0 population)
tribes = covidf_town.loc[covidf_town['pop']==0,'city_name'].unique()## len == 10
covidf_town = covidf_town.loc[~covidf_town.city_name.isin(tribes)]
covidf_town['pop'] = covidf_town['pop'].astype('int')

##SET City_Type
## City >=100,000 ; Town >=10,0000; Village < 10,000
covidf_town['city_type'] = ['City' if x >=100000 else 'Town' if x >=10000 else 'Village' for x in covidf_town['pop']]
# covidf_town[['pop']].plot.hist(bins=10)

#norm coefficients
norm_by_x = lambda x: covidf_town['pop']/x
covidf_town = covidf_town.assign(norm100k=norm_by_x(10**5), norm10k=norm_by_x(10**4), norm1k=norm_by_x(10**4))

##add city type to main COVIDF
covidf = covidf.merge(covidf_town[['city_code','city_type', 'norm100k', 'norm10k', 'norm1k']], on='city_code', how='left')

# len(covidf_town) ##261

In [8]:
covidf.columns

Index(['_id', 'city_name', 'city_code', 'date', 'cumulative_verified_cases',
       'cumulated_recovered', 'cumulated_deaths', 'cumulated_number_of_tests',
       'cumulated_number_of_diagnostic_tests', 'pop', 'city_type', 'norm100k',
       'norm10k', 'norm1k'],
      dtype='object')

In [9]:
##TEST
# col = 'cumulated_deaths'; city_type ='Town'

In [10]:
##normalize column for city_type
def barChartDFandNameTotal(covidf=covidf, col='cumulated_deaths', city_type='City'):
    df_val = covidf.query("city_type == '{}'".format(city_type))
    value_name = '{}_total_{}'.format(city_type.lower(), col.replace('cumulated_',''))
    
    #revert city name (hebrew)
    df_val['city_name'] = df_val['city_name'].apply(lambda x:x[::-1])
    #WIDE format
    df_bar = df_val.pivot(index='date',columns='city_name', values=col)
    # #clean and remove all zeros
    df_bar.columns.name = ''
    df_bar = df_bar.loc[df_bar.apply(lambda x: x.sum(), axis=1) > 0]

    print(value_name)
    print("number of {}: {}".format(city_type, len(df_bar.columns)))
    
    return df_bar, value_name

# df_c = df_p.dropna(how='all').dropna(how='all', axis=1).bfill().ffill()

# df_c.head()


In [11]:
##normalize column for city_type
def barChartDFandNameNorm(covidf=covidf, col='cumulated_deaths', city_type='City'):
    df_val = covidf.query("city_type == '{}'".format(city_type))
    k_col = 'norm100k' if city_type == 'City' else 'norm10k' if city_type == 'Town' else 'norm1k'
    df_val= df_val.assign(value=df_val[col] / df_val[k_col])
    value_name = '{}_{}_per_{}'.format(city_type.lower(), col.replace('cumulated_',''),k_col.replace('norm',''))
    
    #revert city name (hebrew)
    df_val['city_name'] = df_val['city_name'].apply(lambda x:x[::-1])
    #WIDE format
    df_bar = df_val.pivot(index='date',columns='city_name', values='value')
    # #clean and remove all zeros
    df_bar.columns.name = ''
    df_bar = df_bar.loc[df_bar.apply(lambda x: x.sum(), axis=1) > 0]

    print(value_name)
    print("number of {}: {}".format(city_type, len(df_bar.columns)))
    
    return df_bar, value_name

# df_c = df_p.dropna(how='all').dropna(how='all', axis=1).bfill().ffill()

# df_c.head()


In [15]:
def barChartRaceCreator(df_bar, value_name, city_type, n_bars=16):
    title_name = lambda x: x.title().replace('_', ' ')
    total_title = value_name.title().replace('_',' ').replace('{} '.format(city_type), '')
    bcr.bar_chart_race(df_bar,
                   n_bars=n_bars,
                   figsize=(7,5),
                   filter_column_colors=True,
                   period_fmt='%B %d, %Y',
                   period_label={'x': .98, 'y': .2, 'ha': 'right', 'va': 'center',
                                 'family':'Lato','weight':'bold', 'size':14},
                   period_summary_func=lambda v, r: {'x': .98, 'y': .13,
                                      's': '{}: {:,.0f}'.format(total_title, v.nlargest(n_bars).sum()),
                                       'family':'Lato', 'ha': 'right', 'size': 10},
                   filename='barace/{}.mp4'.format(value_name),
                    shared_fontdict={'family' : 'Open Sans Hebrew','weight':'normal'},
                   period_length=200,
                   title = 'COVID in Israel - {}'.format(title_name(value_name)),
                   title_size=11,
                   tick_label_size=10,
                   bar_label_size=11,
                   cmap='tab20')
    

In [16]:
##rcParams - mpl style
mpl.rcParams['font.serif'] = ['Lato']
mpl.rcParams['axes.titleweight'] ='bold'


In [17]:
# ###TEST
# df_bar, value_name = barChartDFandNameNorm(city_type='City')
# df_bar_test = df_bar.loc[df_bar.index.month.isin([4])]
# barChartRaceCreator(df_bar_test, value_name, city_type='City')



In [ ]:
## CREATE NORM AND TOTAL BAR CHARTS for City, Village and Town - this may take sometime...
for city_type in ['City','Town','Village']:
    print(city_type)
    df_bar, value_name = barChartDFandNameNorm(city_type=city_type)
    barChartRaceCreator(df_bar, value_name, city_type)
    df_bar, value_name = barChartDFandNameTotal(city_type=city_type)
    barChartRaceCreator(df_bar, value_name, city_type)


City
city_deaths_per_100k
number of City: 16


C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


city_total_deaths
number of City: 16
Town
town_deaths_per_10k
number of Town: 120


C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214

C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


town_total_deaths
number of Town: 120


C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\eliav\Anaconda3\envs\bcr\lib\site-packages\matplotlib\backends\backend_agg.py:214

### Current Situation

In [ ]:
latest_date = covidf.date.max()
print(latest_date)


In [ ]:
df_latest = covidf.loc[covidf.date==latest_date]
df_city = df_latest.loc[df_latest.city_type=='City']

In [ ]:
df_city = df_city.assign(norm_death_100k=df_city.cumulated_deaths/df_city.norm100k)
df_city.sort_values(by='norm_death_100k',inplace=True)

In [ ]:
df_city.head()

In [ ]:
fig, ax = plt.subplots(figsize=(7,5), dpi=300)
ax.barh(df_city['city_name'].apply(lambda x:x[::-1]), df_city.norm_death_100k)
ax.set_xlabel('הרוגים פר מאה אלף תושבים'[::-1], weight='bold', fontsize=14)
plt.grid(True, axis='x')
ax.set_axisbelow(True)
ax.text(x=.95,y=.1,s=latest_date.strftime("%B %d, %Y"),weight='bold',fontsize=16,ha='right', transform=ax.transAxes)
ax.set_title("מספר הרוגים פר מאה אלף תושבים בערים גדולות"[::-1], weight='bold', fontsize=16)

rects = ax.patches

# Make some labels.
labels = ["label%d" % i for i in np.arange(len(rects))]

for i, rect in enumerate(rects):
    width = rect.get_width()
    reclabel = int(width)
    ax.text(width, i, reclabel,
            ha='left', va='center')

plt.show()
fig.savefig('graphs/town/latest_deaths_norm_city.jpg')
